In [1]:
import numpy as np
import sys
print(sys.executable)
from mpl_toolkits import mplot3d


import deepxde as dde

# 用于绘图
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

a = 1
b = 2
k_r = 1.5
k_i = 1
v_1 = 1
v_2 = 1
v_3 = 1
c = np.log((a * a * v_1 + b * b * v_2 + a * b * v_3 + a * b * v_3) / ((2 * k_r) * (2 * k_r)))
print(sys.executable)
#周期性边界条件
x_lower = -10
x_upper = 10
t_lower = -2
t_upper =2

# 创建 2D 域（用于绘图和输入）
x = np.linspace(x_lower, x_upper, 512)
t = np.linspace(t_lower, t_upper, 402)
X, T = np.meshgrid(x, t)

# 整个域变平
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

# 空间和时间域/几何（对于 deepxde 模型）
space_domain = dde.geometry.Interval(x_lower, x_upper)
time_domain = dde.geometry.TimeDomain(t_lower, t_upper)
geomtime = dde.geometry.GeometryXTime(space_domain, time_domain)
# 损失的“物理信息”部分


def pde(x,y):
    """
    INPUTS:
        x: x[:,0] 是 x 坐标
           x[:,1] 是 t 坐标
        y: 网络输出，在这种情况下:
            y[:,0] 是 u(x,t) 实部
            y[:,1] 是 v(x,t) 虚部
    OUTPUT:
        标准形式的 pde，即必须为零的东西
    """
    
    
    u1 = y[:, 0:1]
    v1 = y[:, 1:2]
    u2 = y[:, 2:3]
    v2 = y[:, 3:4]
    

    # 在'jacobian'中，i 是输出分量，j 是输入分量
    u1_t = dde.grad.jacobian(y, x, i=0, j=1)
    v1_t = dde.grad.jacobian(y, x, i=1, j=1)
    u2_t = dde.grad.jacobian(y, x, i=2, j=1)
    v2_t = dde.grad.jacobian(y, x, i=3, j=1)

    u1_x = dde.grad.jacobian(y, x, i=0, j=0)
    v1_x = dde.grad.jacobian(y, x, i=1, j=0)
    u2_x = dde.grad.jacobian(y, x, i=2, j=0)
    v2_x = dde.grad.jacobian(y, x, i=3, j=0)

    # 在“hessian”中，i 和 j 都是输入分量。 （Hessian 原则上可以是 d^2y/dxdt、d^2y/d^2x 等）
    # 输出组件由“组件”选择
    u1_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    v1_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    u2_xx = dde.grad.hessian(y, x, component=2, i=0, j=0)
    v2_xx = dde.grad.hessian(y, x, component=3, i=0, j=0)

    #f_u = u_t + 0.5 * v_xx + (u ** 2 + v ** 2) * v
    #f_v = v_t - 0.5 * u_xx - (u ** 2 + v ** 2) * u

    r1=2
    r2=4
    r3=-2
    r4=0
    beta=1
    
    f_u1 = (
            v1_t + u1_xx
        + r1*u1*(u1**2 + v1**2) + r2*u1*(u2**2 + v2**2) + r3*(u1*(u2**2 - v2**2) + 2*u2*v1*v2) + r4*(u2*(u1**2 - v1**2) + 2*u1*v1*v2)
    )

    f_v1 = (
            -u1_t + v1_xx
        + r1*v1*(u1**2 + v1**2) + r2*v1*(u2**2 + v2**2) - r3*(v1*(u2**2 - v2**2) - 2*u1*u2*v2) - r4*(v2*(u1**2 - v1**2) - 2*u1*u2*v1)
    )

    f_u2 = (
            v2_t + beta*u1_xx
        + r1*u2*(u2**2 + v2**2) + r2*u2*(u1**2 + v1**2) + r3*(u2*(u1**2 - v1**2) + 2*u1*v2*v1) + r4*(u1*(u2**2 - v2**2) + 2*u2*v2*v1)
    )

    f_v2 = (
            -u2_t + beta*v2_xx
        + r1*v2*(u2**2 + v2**2) + r2*v2*(u1**2 + v1**2) - r3*(v2*(u1**2 - v1**2) - 2*u2*u1*v1) - r4*(v1*(u2**2 - v2**2) - 2*u2*u1*v2)
    )
    
    return [
        f_u1, f_v1, f_u2, f_v2,
        #f_u1x, f_u1t,
        #f_v1x, f_v1t,
        #f_u2x, f_u2t,
        #f_v2x, f_v2t
    ]
 # 边界条件和初始条件

# 周期性边界条件
bc_u1_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=0
)
bc_u1_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=0
)
bc_v1_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=1
)
bc_v1_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=1
)
bc_u2_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=2
)
bc_u2_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=2
)
bc_v2_0 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=3
)
bc_v2_1 = dde.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=3
)
#(e^c=...)
# 初始条件
def boundary_l(x, on_boundary):
    return on_boundary and np.isclose(x[0], -1)

m0=1 #a
m1=np.sqrt(1) #b
m2=0 #Δ
m3=0 #θ





#lambda
c11=1
c21=2
c31=-2
c41=4
lambda1=0.01-1j

v1=(c31.conjugate()**2+c41.conjugate()**2)*(c11*c31+c21*c41)
v2=(c11**2+c21**2)*(c11.conjugate()**2+c21.conjugate()**2)
v3=(c31.conjugate()**2+c41.conjugate()**2)*(c21*c31-c11*c41)

eta1=np.log((c11**2+c21**2)*(c11.conjugate()*c31.conjugate()+c21.conjugate()*c41.conjugate())/v1)/2
eta2=np.log(v2/((c11*c11.conjugate()+c21*c21.conjugate())*(c31*c31.conjugate()+c41*c41.conjugate())))/2
eta3=np.log((c11**2+c21**2)*(c21*c31-c11*c41)/v3)/2
gamma1=(c31**2+c41**2)*(c31.conjugate()**2+c41.conjugate()**2)

def init_cond_u1(x):
    xi1=-2*1j*lambda1**2*x[:,1:2]-1j*lambda1*x[:,0:1]
    q1=4*v1*1j*(lambda1.conjugate()-lambda1)*np.exp(3*xi1+xi1.conjugate()+eta1)*np.cosh(xi1+xi1.conjugate()+eta1)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    q2=4*v3*1j*(lambda1-lambda1.conjugate())*np.exp(3*xi1+xi1.conjugate()+eta3)*np.cosh(xi1+xi1.conjugate()+eta3)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    return q1.real
    
def init_cond_u2(x):
    "2 tanh(x)"
    xi1=-2*1j*lambda1**2*x[:,1:2]-1j*lambda1*x[:,0:1]
    q1=4*v1*1j*(lambda1.conjugate()-lambda1)*np.exp(3*xi1+xi1.conjugate()+eta1)*np.cosh(xi1+xi1.conjugate()+eta1)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    q2=4*v3*1j*(lambda1-lambda1.conjugate())*np.exp(3*xi1+xi1.conjugate()+eta3)*np.cosh(xi1+xi1.conjugate()+eta3)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    return q2.real

def init_cond_v1(x):
    xi1=-2*1j*lambda1**2*x[:,1:2]-1j*lambda1*x[:,0:1]
    q1=4*v1*1j*(lambda1.conjugate()-lambda1)*np.exp(3*xi1+xi1.conjugate()+eta1)*np.cosh(xi1+xi1.conjugate()+eta1)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    q2=4*v3*1j*(lambda1-lambda1.conjugate())*np.exp(3*xi1+xi1.conjugate()+eta3)*np.cosh(xi1+xi1.conjugate()+eta3)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    return -q1.imag

def init_cond_v2(x):
    xi1=-2*1j*lambda1**2*x[:,1:2]-1j*lambda1*x[:,0:1]
    q1=4*v1*1j*(lambda1.conjugate()-lambda1)*np.exp(3*xi1+xi1.conjugate()+eta1)*np.cosh(xi1+xi1.conjugate()+eta1)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    q2=4*v3*1j*(lambda1-lambda1.conjugate())*np.exp(3*xi1+xi1.conjugate()+eta3)*np.cosh(xi1+xi1.conjugate()+eta3)/(2*v2*np.exp(3*(xi1+xi1.conjugate())+eta2)*np.cosh(xi1+xi1.conjugate()+eta2)+gamma1)
    return -q2.imag

ic_u1 = dde.IC(geomtime, init_cond_u1, lambda _, on_initial: on_initial, component=0)
ic_v1 = dde.IC(geomtime, init_cond_v1, lambda _, on_initial: on_initial, component=1)
ic_u2 = dde.IC(geomtime, init_cond_u2, lambda _, on_initial: on_initial, component=2)
ic_v2 = dde.IC(geomtime, init_cond_v2, lambda _, on_initial: on_initial, component=3)

data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc_u1_0, bc_u1_1, bc_v1_0, bc_v1_1,bc_u2_0, bc_u2_1, bc_v2_0, bc_v2_1, ic_u1, ic_v1,ic_u2, ic_v2],
    num_domain=10000,
    num_boundary=20,
    num_initial=200,
    train_distribution="pseudo",
)

allow_unused=True
# 网络架构
net = dde.maps.FNN([2] + [100] * 5 + [4], "silu", "Glorot normal")

model = dde.Model(data, net)

D:\anaconda\python.exe


Using backend: tensorflow.compat.v1



Instructions for updating:
non-resource variables are not supported in the long term
Enable just-in-time compilation with XLA.


D:\anaconda\python.exe


In [2]:
model.compile("adam", lr=1e-3, loss="MSE")
model.train(epochs=3000, display_every=10)

Compiling model...
Building feed-forward neural network...
'build' took 0.185705 s



C:\Users\86183\AppData\Roaming\Python\Python39\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:103: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 13.205659 s

Initializing variables...
Training model...

Step      Train loss                                                                                                                                                          Test loss                                                                                                                                                           Test metric
0         [7.28e-05, 2.29e-05, 9.33e-05, 1.08e-04, 8.44e-03, 3.43e-05, 5.20e-02, 1.19e-05, 4.78e-04, 8.81e-06, 5.53e-02, 1.85e-03, 1.42e-02, 6.90e-01, 2.57e-02, 1.19e+00]    [7.28e-05, 2.29e-05, 9.33e-05, 1.08e-04, 8.44e-03, 3.43e-05, 5.20e-02, 1.19e-05, 4.78e-04, 8.81e-06, 5.53e-02, 1.85e-03, 1.42e-02, 6.90e-01, 2.57e-02, 1.19e+00]    []  
10        [3.09e-04, 8.96e-05, 3.10e-03, 5.61e-05, 3.84e-04, 4.20e-05, 6.56e-03, 5.49e-04, 4.30e-04, 5.42e-05, 1.16e-03, 1.01e-03, 1.31e-02, 6.24e-01, 2.21e-02, 1.06e+00]    [3.09e-04, 8.96e-05, 3.10e-03, 5.61e-05, 3.84e-04, 4.20e-05,

230       [1.81e-02, 2.13e-02, 2.73e-02, 3.98e-02, 4.23e-05, 1.57e-06, 8.78e-06, 2.95e-05, 4.84e-05, 1.37e-05, 2.39e-06, 2.03e-05, 5.78e-03, 2.74e-01, 1.06e-02, 4.93e-01]    [1.81e-02, 2.13e-02, 2.73e-02, 3.98e-02, 4.23e-05, 1.57e-06, 8.78e-06, 2.95e-05, 4.84e-05, 1.37e-05, 2.39e-06, 2.03e-05, 5.78e-03, 2.74e-01, 1.06e-02, 4.93e-01]    []  
240       [1.96e-02, 2.04e-02, 3.18e-02, 3.76e-02, 6.69e-07, 7.56e-06, 2.47e-06, 2.00e-05, 3.04e-06, 3.07e-06, 2.35e-06, 1.72e-05, 4.79e-03, 2.72e-01, 8.42e-03, 4.90e-01]    [1.96e-02, 2.04e-02, 3.18e-02, 3.76e-02, 6.69e-07, 7.56e-06, 2.47e-06, 2.00e-05, 3.04e-06, 3.07e-06, 2.35e-06, 1.72e-05, 4.79e-03, 2.72e-01, 8.42e-03, 4.90e-01]    []  
250       [1.77e-02, 2.01e-02, 2.88e-02, 3.71e-02, 9.43e-06, 9.32e-06, 5.45e-06, 1.05e-05, 2.51e-05, 2.21e-06, 1.77e-06, 1.06e-05, 6.24e-03, 2.70e-01, 1.07e-02, 4.88e-01]    [1.77e-02, 2.01e-02, 2.88e-02, 3.71e-02, 9.43e-06, 9.32e-06, 5.45e-06, 1.05e-05, 2.51e-05, 2.21e-06, 1.77e-06, 1.06e-05, 6.24e-03, 2.70e-01,

470       [2.39e-02, 1.45e-02, 3.75e-02, 2.86e-02, 7.25e-06, 3.72e-06, 1.95e-06, 1.27e-05, 1.94e-06, 8.42e-06, 4.61e-07, 5.66e-06, 5.90e-03, 2.43e-01, 9.69e-03, 4.42e-01]    [2.39e-02, 1.45e-02, 3.75e-02, 2.86e-02, 7.25e-06, 3.72e-06, 1.95e-06, 1.27e-05, 1.94e-06, 8.42e-06, 4.61e-07, 5.66e-06, 5.90e-03, 2.43e-01, 9.69e-03, 4.42e-01]    []  
480       [2.41e-02, 1.44e-02, 3.79e-02, 2.83e-02, 7.11e-06, 3.81e-06, 1.85e-06, 1.44e-05, 1.13e-06, 6.99e-06, 4.36e-07, 7.37e-06, 5.94e-03, 2.41e-01, 9.74e-03, 4.40e-01]    [2.41e-02, 1.44e-02, 3.79e-02, 2.83e-02, 7.11e-06, 3.81e-06, 1.85e-06, 1.44e-05, 1.13e-06, 6.99e-06, 4.36e-07, 7.37e-06, 5.94e-03, 2.41e-01, 9.74e-03, 4.40e-01]    []  
490       [2.44e-02, 1.43e-02, 3.82e-02, 2.81e-02, 6.63e-06, 3.55e-06, 1.76e-06, 1.60e-05, 7.03e-07, 5.53e-06, 4.28e-07, 9.39e-06, 5.98e-03, 2.40e-01, 9.76e-03, 4.37e-01]    [2.44e-02, 1.43e-02, 3.82e-02, 2.81e-02, 6.63e-06, 3.55e-06, 1.76e-06, 1.60e-05, 7.03e-07, 5.53e-06, 4.28e-07, 9.39e-06, 5.98e-03, 2.40e-01,

...

2150      [1.31e-02, 2.22e-02, 1.87e-02, 4.07e-02, 6.97e-06, 3.09e-05, 6.00e-07, 8.47e-06, 5.99e-06, 1.90e-05, 4.72e-06, 1.70e-05, 5.46e-03, 2.70e-01, 1.01e-02, 4.87e-01]    [1.31e-02, 2.22e-02, 1.87e-02, 4.07e-02, 6.97e-06, 3.09e-05, 6.00e-07, 8.47e-06, 5.99e-06, 1.90e-05, 4.72e-06, 1.70e-05, 5.46e-03, 2.70e-01, 1.01e-02, 4.87e-01]    []  
2160      [1.35e-02, 2.16e-02, 1.93e-02, 4.00e-02, 6.38e-06, 3.39e-05, 7.25e-07, 9.52e-06, 4.70e-06, 2.10e-05, 2.88e-06, 1.96e-05, 5.41e-03, 2.68e-01, 1.01e-02, 4.84e-01]    [1.35e-02, 2.16e-02, 1.93e-02, 4.00e-02, 6.38e-06, 3.39e-05, 7.25e-07, 9.52e-06, 4.70e-06, 2.10e-05, 2.88e-06, 1.96e-05, 5.41e-03, 2.68e-01, 1.01e-02, 4.84e-01]    []  
2170      [1.39e-02, 2.11e-02, 1.98e-02, 3.92e-02, 6.06e-06, 3.67e-05, 1.09e-06, 1.06e-05, 3.30e-06, 2.27e-05, 1.51e-06, 2.28e-05, 5.37e-03, 2.66e-01, 1.01e-02, 4.80e-01]    [1.39e-02, 2.11e-02, 1.98e-02, 3.92e-02, 6.06e-06, 3.67e-05, 1.09e-06, 1.06e-05, 3.30e-06, 2.27e-05, 1.51e-06, 2.28e-05, 5.37e-03, 2.66e-01,

2390      [1.68e-02, 1.76e-02, 2.56e-02, 3.22e-02, 3.99e-06, 1.47e-05, 5.17e-06, 1.34e-05, 2.51e-06, 1.25e-06, 2.88e-06, 4.33e-05, 4.93e-03, 2.34e-01, 9.30e-03, 4.34e-01]    [1.68e-02, 1.76e-02, 2.56e-02, 3.22e-02, 3.99e-06, 1.47e-05, 5.17e-06, 1.34e-05, 2.51e-06, 1.25e-06, 2.88e-06, 4.33e-05, 4.93e-03, 2.34e-01, 9.30e-03, 4.34e-01]    []  
2400      [1.26e-02, 1.33e-02, 1.69e-02, 2.43e-02, 9.30e-06, 1.11e-05, 5.66e-06, 1.41e-05, 1.73e-05, 1.33e-06, 7.53e-06, 4.24e-05, 8.28e-03, 2.43e-01, 1.62e-02, 4.48e-01]    [1.26e-02, 1.33e-02, 1.69e-02, 2.43e-02, 9.30e-06, 1.11e-05, 5.66e-06, 1.41e-05, 1.73e-05, 1.33e-06, 7.53e-06, 4.24e-05, 8.28e-03, 2.43e-01, 1.62e-02, 4.48e-01]    []  
2410      [4.16e-02, 2.67e-02, 3.74e-02, 5.36e-02, 4.71e-04, 3.17e-05, 1.11e-05, 1.30e-05, 6.14e-04, 2.02e-06, 1.06e-06, 5.18e-05, 1.52e-03, 2.28e-01, 4.53e-03, 4.21e-01]    [4.16e-02, 2.67e-02, 3.74e-02, 5.36e-02, 4.71e-04, 3.17e-05, 1.11e-05, 1.30e-05, 6.14e-04, 2.02e-06, 1.06e-06, 5.18e-05, 1.52e-03, 2.28e-01,

2630      [2.25e-02, 1.70e-02, 2.01e-02, 3.77e-02, 8.08e-06, 5.45e-06, 6.46e-06, 1.31e-05, 4.72e-05, 1.57e-05, 2.25e-05, 1.79e-05, 2.59e-03, 2.12e-01, 7.25e-03, 3.86e-01]    [2.25e-02, 1.70e-02, 2.01e-02, 3.77e-02, 8.08e-06, 5.45e-06, 6.46e-06, 1.31e-05, 4.72e-05, 1.57e-05, 2.25e-05, 1.79e-05, 2.59e-03, 2.12e-01, 7.25e-03, 3.86e-01]    []  
2640      [1.82e-02, 1.02e-02, 2.28e-02, 2.35e-02, 9.27e-06, 1.34e-06, 1.12e-05, 2.27e-05, 3.69e-06, 3.20e-06, 1.37e-05, 4.58e-05, 2.68e-03, 2.15e-01, 6.84e-03, 3.94e-01]    [1.82e-02, 1.02e-02, 2.28e-02, 2.35e-02, 9.27e-06, 1.34e-06, 1.12e-05, 2.27e-05, 3.69e-06, 3.20e-06, 1.37e-05, 4.58e-05, 2.68e-03, 2.15e-01, 6.84e-03, 3.94e-01]    []  
2650      [1.40e-02, 1.24e-02, 2.32e-02, 2.45e-02, 2.80e-06, 5.03e-06, 2.89e-06, 1.69e-05, 9.29e-06, 3.58e-06, 1.58e-05, 1.43e-05, 3.94e-03, 2.10e-01, 8.39e-03, 3.89e-01]    [1.40e-02, 1.24e-02, 2.32e-02, 2.45e-02, 2.80e-06, 5.03e-06, 2.89e-06, 1.69e-05, 9.29e-06, 3.58e-06, 1.58e-05, 1.43e-05, 3.94e-03, 2.10e-01,

2870      [1.60e-02, 1.56e-02, 2.15e-02, 2.83e-02, 2.39e-05, 1.53e-05, 1.14e-05, 1.38e-05, 8.76e-06, 1.80e-05, 2.54e-05, 1.70e-05, 3.58e-03, 1.97e-01, 7.41e-03, 3.52e-01]    [1.60e-02, 1.56e-02, 2.15e-02, 2.83e-02, 2.39e-05, 1.53e-05, 1.14e-05, 1.38e-05, 8.76e-06, 1.80e-05, 2.54e-05, 1.70e-05, 3.58e-03, 1.97e-01, 7.41e-03, 3.52e-01]    []  
2880      [1.61e-02, 1.61e-02, 2.16e-02, 2.86e-02, 1.18e-05, 9.83e-06, 4.73e-06, 8.76e-06, 1.04e-05, 1.48e-05, 4.23e-05, 1.20e-05, 3.55e-03, 1.93e-01, 7.53e-03, 3.46e-01]    [1.61e-02, 1.61e-02, 2.16e-02, 2.86e-02, 1.18e-05, 9.83e-06, 4.73e-06, 8.76e-06, 1.04e-05, 1.48e-05, 4.23e-05, 1.20e-05, 3.55e-03, 1.93e-01, 7.53e-03, 3.46e-01]    []  
2890      [1.55e-02, 1.67e-02, 2.18e-02, 2.93e-02, 6.69e-06, 9.01e-06, 1.74e-06, 1.05e-05, 6.93e-06, 1.20e-05, 4.86e-05, 5.73e-06, 3.60e-03, 1.89e-01, 7.55e-03, 3.41e-01]    [1.55e-02, 1.67e-02, 2.18e-02, 2.93e-02, 6.69e-06, 9.01e-06, 1.74e-06, 1.05e-05, 6.93e-06, 1.20e-05, 4.86e-05, 5.73e-06, 3.60e-03, 1.89e-01,

(<deepxde.model.LossHistory at 0x188ed6d8fd0>,
 <deepxde.model.TrainState at 0x188ed6d87f0>)

In [ ]:
dde.optimizers.config.set_LBFGS_options(
    maxcor=50,
    ftol=1.0 * np.finfo(float).eps,
    gtol=1e-08,
    maxiter=3000,
    maxfun=5000,
    maxls=50,
)
model.compile("L-BFGS")
#model.train()

losshistory, train_state = model.train()


dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 8.231612 s

Training model...

Step      Train loss                                                                                                                                                          Test loss                                                                                                                                                           Test metric
3000      [1.57e-02, 1.94e-02, 2.15e-02, 3.18e-02, 3.23e-06, 8.66e-06, 2.01e-06, 2.21e-05, 7.22e-06, 7.97e-06, 6.26e-07, 3.06e-06, 3.23e-03, 1.65e-01, 7.47e-03, 3.06e-01]    [1.57e-02, 1.94e-02, 2.15e-02, 3.18e-02, 3.23e-06, 8.66e-06, 2.01e-06, 2.21e-05, 7.22e-06, 7.97e-06, 6.26e-07, 3.06e-06, 3.23e-03, 1.65e-01, 7.47e-03, 3.06e-01]    []  


In [ ]:

loss_t = losshistory.loss_train



plt.plot(np.mean(loss_t,axis=1))
plt.xlabel("step")
plt.ylabel("loss")

print(losshistory)
print(train_state)

In [ ]:
# 做预测
prediction = model.predict(X_star, operator=None)

u1 = griddata(X_star, prediction[:, 0], (X, T), method="cubic")
v1 = griddata(X_star, prediction[:, 1], (X, T), method="cubic")
u2 = griddata(X_star, prediction[:, 2], (X, T), method="cubic")
v2 = griddata(X_star, prediction[:, 3], (X, T), method="cubic")


h1 = np.sqrt(u1 ** 2 + v1 ** 2)
h2=np.sqrt(u2**2+v2**2)



# 绘制预测
fig, ax = plt.subplots(3)

ax[0].set_title("Results1")
ax[0].set_ylabel("Real part1")
ax[0].imshow(
    u1.T,
    interpolation="nearest", #插值
    cmap="viridis", #色彩图
    extent=[t_lower, t_upper, x_lower, x_upper], #极值
    origin="lower",
    aspect="auto",
)
ax[1].set_ylabel("Imaginary part1")
ax[1].imshow(
    v1.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
ax[2].set_ylabel("Amplitude1")
ax[2].imshow(
    h1.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)

fig, bx = plt.subplots(3)
bx[0].set_title("Results2")
bx[0].set_ylabel("Real part2")
bx[0].imshow(
    u2.T,
    interpolation="nearest", #插值
    cmap="viridis", #色彩图
    extent=[t_lower, t_upper, x_lower, x_upper], #极值
    origin="lower",
    aspect="auto",
)
bx[1].set_ylabel("Imaginary part2")
bx[1].imshow(
    v2.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
bx[2].set_ylabel("Amplitude1")
bx[2].imshow(
    h2.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
plt.show()

In [ ]:
ax = plt.axes(projection='3d')


b=prediction[:,0]**2+prediction[:,1]**2
c=min(b)
d=b/c
d=d.astype(int)
print(d)
# Data for a three-dimensional line
ax.scatter3D(X_star[:,0],X_star[:,1],b,c=d[:],cmap='plasma')


In [ ]:
bx = plt.axes(projection='3d')
b=prediction[:,2]**2+prediction[:,3]**2
c=min(b)
d=b/c
d=d.astype(int)
print(d)
bx.scatter3D(X_star[:,0],X_star[:,1],b,c=d[:]/10)

print(X.shape)
print(T.shape)
print(X_star.shape)
print(prediction.shape)

In [ ]:
print(c)